In [1]:
# ------- SERVER EXTENSIONS ---------
lib =  r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities/droplet_dataset'
lib2 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities'
lib3 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/data_analysis'
lib4 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy'
lib5 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/scripts'
import sys
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)
# ------- SERVER EXTENSIONS ---------import numpy as np
import pandas as pd
import scipy
import sklearn
from sklearn.manifold import TSNE
import pickle
# from Bio.Cluster import kcluster
import os
import numpy as np
import yaml
import os
import pandas
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sys
import seaborn as sns
# import pyclustering
import matplotlib.pylab as plt
import seaborn as sb
from os.path import join
plt.rcParams['figure.dpi'] = 400
import seaborn as sns
from utilities.general_helpers import sort_dic
from utilities.general_helpers import transpose
from general_helpers import create_folder
from shutil import copyfile
import matplotlib.pyplot as plt
import random


In [5]:
# Loads xlsx files
CLINICAL_LABELS_PATH = r'/storage/md_keren/shitay/Data/tables/clinical_labels.xlsx'
MELANOMA_CLINICAL_DATA_PATH = r'/storage/md_keren/shitay/Data/tables/edited_unprotected_Melanoma_clinical_data.xlsx'
melanoma_clinical_data = pd.read_excel(MELANOMA_CLINICAL_DATA_PATH)
clinical_labels = pd.read_excel(CLINICAL_LABELS_PATH)

# takes nly first 46 samples, fill Nan and creat dictionary mapping
melanoma_clinical_data = melanoma_clinical_data.iloc[:46,:][['Patient id', 'Clinical response']]
melanoma_clinical_data['Clinical response'] = melanoma_clinical_data['Clinical response'].fillna('??')
labels_mapping = {v[0]: v[1] for v in clinical_labels[['Clinical response', 'binary label']].values}

# adds CRITERIA response into clinical df
melanoma_clinical_data['response'] = melanoma_clinical_data['Clinical response'].apply(lambda x: labels_mapping[x])

In [60]:
melanoma_clinical_data.head(5)


,Patient id,Clinical response,response
0,M97,R,R
1,M98,R (had a response in 2016-2019),R
2,M99,NR (PD),NR
3,M100,NR (SD),NR
4,M101,NED- 10/5/20,R


In [34]:
response_samples = melanoma_clinical_data[melanoma_clinical_data['response'] == 'R']['Patient id'].tolist()
non_response_samples = melanoma_clinical_data[melanoma_clinical_data['response'] == 'NR']['Patient id'].tolist()
no_used_samples = melanoma_clinical_data[melanoma_clinical_data['response'] == 'not in use for now']['Patient id'].tolist()

print(f'number reponse_samples: {len(reponse_samples)}')
print(f'number non_reponse_samples: {len(non_reponse_samples)}')
print(f'number non_reponse_samples: {len(no_used_samples)}')

number reponse_samples: 14
number non_reponse_samples: 25
number non_reponse_samples: 7


#### Loads tumor cohort:

In [61]:
# tumor_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/cohort_normalized_24.5.21.pkl'
COHORT_PATH = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/cohort_normalized_26.6.21.pkl'

# tumor_cohort = pickle.load(open(tumor_path, 'rb'))
cohort = pickle.load(open(COHORT_PATH, 'rb'))
tumor_cohort = cohort.filter_cells_by_property('is_cancer', True)
immune_cohort = cohort.filter_cells_by_property('is_immune', True)



# Diffrenital analysis

In [58]:
response_samples_indices = [s in response_samples for s in tumor_cohort.samples]
non_response_samples_indices =[s in non_response_samples for s in tumor_cohort.samples]
no_used_samples_indices = [s in no_used_samples for s in tumor_cohort.samples]

print(f'number of cells in response_samples_indices: {sum(response_samples_indices)}')
print(f'number of cells in non_response_samples_indices: {sum(non_response_samples_indices)}')
print(f'number of cells in no_used_samples_indices: {sum(no_used_samples_indices)}')

response_samples_cells = tumor_cohort[response_samples_indices]
non_response_samples_cells = tumor_cohort[non_response_samples_indices]


number of cells in response_samples_indices: 14818
number of cells in non_response_samples_indices: 64219
number of cells in no_used_samples_indices: 25898


In [59]:
14818 + 64219 + 25898

104935

In [44]:
response_samples_cells

14818

In [56]:
# from utilities.clustering_tools import find_marker_genes_in_cluster

res = find_marker_genes_in_cluster(response_samples_cells, non_response_samples_cells, log_FC_threshold=0.05, pval_threshold=0.05, min_pct=0.1, min_diff_pct=0.1)

In [57]:
res

,features,gene names,(1)mean_expression,(2)mean expression,log_FC,(1)#expressing,(2)#expressing,(1)%expressing,(2)%expressing,%expressing_diff,pvals
820,ENSG00000060718,COL11A1,0.292458,0.009934,3.923403,1992,286,0.134431,0.004454,0.129978,0.000000e+00
10213,ENSG00000158639,PAGE5,0.906330,0.070841,3.502706,5102,2033,0.344311,0.031657,0.312654,0.000000e+00
13818,ENSG00000175600,SUGCT,0.423747,0.029569,3.454415,2366,589,0.159671,0.009172,0.150499,0.000000e+00
18093,ENSG00000205362,MT1A,0.478224,0.050748,3.006630,3214,1205,0.216898,0.018764,0.198134,0.000000e+00
157,ENSG00000006747,SCIN,0.366591,0.037441,2.988794,2563,927,0.172965,0.014435,0.158530,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...
14715,ENSG00000180879,SSR4,1.773674,1.568899,0.175933,10531,38916,0.710690,0.605989,0.104701,4.569753e-128
12589,ENSG00000169714,CNBP,1.813595,1.615075,0.166279,10993,40656,0.741868,0.633084,0.108784,5.463446e-144
17365,ENSG00000198668,CALM1,1.874649,1.674842,0.161683,10787,40096,0.727966,0.624364,0.103602,1.439610e-128
9271,ENSG00000150459,SAP18,1.394187,1.258821,0.146246,9211,33492,0.621609,0.521528,0.100081,1.061168e-108


# Group analysis

In [46]:
# tumor_cohort.samples

# Debug

In [55]:
from scipy.stats import rankdata
import scipy.stats as stats
def multipletests_fdr(p_vals, alpha=0.05):
    """
    Test results and p-value correction for multiple tests using Benjamini/Hochberg method.
    :param p_vals: np.array of pvals
    :return: pvals_corrected
    """

    ranked_p_values = rankdata(p_vals)
    pvals_corrected = p_vals * len(p_vals) / ranked_p_values
    pvals_corrected[pvals_corrected > 1] = 1

    return pvals_corrected < alpha, pvals_corrected


def find_marker_genes_in_cluster(cluster_data, other_clusters_data, log_FC_threshold, pval_threshold,
                                 min_pct, min_diff_pct):
    """
    After the clustering process has been done run this function to find marker genes for each cluster.
    The function conducts a Fisher Exact Test for every gene to check whether that gene constitutes a marker
    of one of the clusters. Obviously, we conduct a test per cluster. that means that if we have k clusters and n
    genes we will have k*n tests.
    We test whether the gene in the cluster expresses differently of the other clusters.
    we define expressed as value > 1 (after normalization). and we test the proportion between the number of cells
    expressing each gene compered to all other clusters together.
    because we conduct many statistical test we will correct the pvalues using Benjamini/Hochberg correction
    :param Data_RNAseq: Can be cohort object or singular RNAseq object which all its cells having association to one of the clusters.
    :param clusters_list: cluster in size of number of cells such that each place indicate which cluster corresponds
    to that cell (number of cluster).
    :param min_diff_pct:
    minimum percent difference between the percent of cells expressing the gene in the cluster
    and the percent of cells expressing gene in all other clusters combined.

    :param min_pct: 0.1
    only test genes that are detected in a minimum fraction of cells in either of the two populations.

    Note: Marker defined as gene with pval < pval_thresh and log_FC < log_ratio_threshold
    :return:
    gene names - of all gene markers, ordered by pval_corrected
    gene ids - of all gene markers, ordered by pval_corrected
    pval - of all gene markers, ordered by pval_corrected
    log ratios - of all gene markers, ordered by pval_corrected
    """

    n_cells_in_cluster = len(cluster_data.counts)
    n_cells_in_other_clusters = len(other_clusters_data.counts)
    min_expression_threshold = 1    # for a cell to be counted as a cell that expresses the gene

    df = pd.DataFrame()
    df['features'] = cluster_data.features
    df['gene names'] = cluster_data.gene_names
    df['(1)mean_expression'] = np.mean(cluster_data.counts, axis=0)
    df['(2)mean expression'] = np.mean(other_clusters_data.counts, axis=0)
    df['log_FC'] = np.log2((df['(1)mean_expression'] + 0.01) / (df['(2)mean expression'] + 0.01))
    df['(1)#expressing'] = np.sum((cluster_data.counts > min_expression_threshold), axis=0)
    df['(2)#expressing'] = np.sum((other_clusters_data.counts > min_expression_threshold), axis=0)
    df['(1)%expressing'] = df['(1)#expressing'] / n_cells_in_cluster
    df['(2)%expressing'] = df['(2)#expressing'] / n_cells_in_other_clusters
    df['%expressing_diff'] = df['(1)%expressing'] - df['(2)%expressing']

    # filters not satisfying genes.
    df = df[(df["(1)%expressing"] > min_pct)]
    df = df[(df["%expressing_diff"] > min_diff_pct)]
    df = df[(df['log_FC'] > log_FC_threshold)]
    df = df.sort_values(['log_FC'], ascending=False)

    p_values = []  # before correction
    for index, row in df.iterrows():
        number_of_cells_expressing_gene_in_cluster = row["(1)#expressing"]
        number_of_cells_expressing_gene_in_other_clusters = row["(2)#expressing"]

        oddsratio, pvalue = stats.fisher_exact([[number_of_cells_expressing_gene_in_cluster,
                                                 number_of_cells_expressing_gene_in_other_clusters],
                                                [n_cells_in_cluster - number_of_cells_expressing_gene_in_cluster,
                                                 n_cells_in_other_clusters - number_of_cells_expressing_gene_in_other_clusters]])

        p_values.append(pvalue)

    reject_arr, pvals_corrected = multipletests_fdr(np.array(p_values), alpha=0.05)

    df['pvals'] = pvals_corrected.tolist()
    df = df[df['pvals'] < pval_threshold]

    return df